In [2]:
import os
import sys
import matplotlib.pyplot as plt
import math
import numpy as np
import pickle
from packages.helper import traj_speed
from packages import data_container
from packages.data_container import Data
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [5]:
header = ('subj_x', 'subj_y', 'subj_yaw')

info = {'goal_angle': [], 'p_goal': [], 'subj_id': [], 'trial_id': [], 'goal_onset': [], 'w_goal': 0.1, 'w_obst': 0.1,
        'ps':[]}
exp1a = Data(Hz=30, header=header, info=info)

data_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\F&W2003 Exp1a rawData'))
ii = 0
goal_dist = 0
goal_angle = 0
goal_side = 0 # +1 means on the right side, -1 means on the left side
p_goal = []
goal_onset = 0
# subj_ps = []
ps = 0

for data_file in os.listdir(data_dir):
    i_subj = int(data_file[-1])
    data_file_path = os.path.join(data_dir, data_file)
    i_trial = 0
    with open(data_file_path, 'r') as f:
        trials = f.read().split('\n')
        head = 4
        length = int(trials[head])
#         trial_ps = []
        while True:
            # import data
            trial = [line.split('\t') for line in trials[head + 1: head + 1 + length]]
            trial = tuple([(float(vals[0])/100, float(vals[1])/100, float(vals[2])) for vals in trial])
            exp1a.add_traj(trial)
            # Use the average speed from the -3 to -2 second as the trial preferred speed
            speed = traj_speed(np.array(exp1a.get_traj(len(exp1a.trajs)-1))[:,:2], exp1a.Hz)
            ps = np.mean(speed[-3*exp1a.Hz : -2*exp1a.Hz])
            # Use the average speed from the -3 to -2 second as the preferred speed
#             trial_ps.append(np.mean(speed[-3*exp1a.Hz : -2*exp1a.Hz]))
            # Use the speed at t1 
            
            # import info
            for i in range(len(trial)):
                if trial[i][1] >= 1:
                    goal_onset = i
                    break
            if trials[head-1].split(' ')[0] == '0':
                cond = trials[head-2].split(' ')
                goal_side = float(cond[2])
                goal_angle = goal_side * float(cond[3]) * 2 * math.pi / 360
                goal_dist = float(cond[4]) / 100
            p_goal = [(goal_dist * math.sin(goal_angle), goal_dist * math.cos(goal_angle)+1)] * length
            p_goal = tuple(p_goal)

            angle = round(goal_angle * 360 / 2 / math.pi)
            exp1a.add_info({'goal_onset':goal_onset, 'goal_angle':angle, 'p_goal':p_goal, 'subj_id':i_subj, 
                            'trial_id':i_trial, 'ps':ps})
            
            if head + length + 3 >= len(trials):
                break
            
            if len(trials[head+length+3].split(' ')) == 1:
                head = head+length+3
            else:
                head = head+length+4
            length = int(trials[head])
            ii += 1
            i_trial += 1
        # Use the average of trial preferred speed as the subject's preferred speed
#         subj_ps.append(np.mean(trial_ps))
    
# Setting subject preferred speed to every trial
# for i in range(len(exp1a.trajs)):
#     i_subj = exp1a.info['subj_id'][i]
#     exp1a.add_info({'ps': subj_ps[i_subj]})

outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Fajen_steer_exp1a_data.pickle'))
with open(outfile, 'wb') as file:   
    pickle.dump(exp1a, file, pickle.HIGHEST_PROTOCOL) 

In [32]:
# Check imported data
vals = []
%matplotlib qt
for i in range(len(exp1a.trajs)):
    val = exp1a.info['ps'][i]
    vals.append(val)
    p_goal = np.array(exp1a.info['p_goal'][i])
    p_subj = np.array(exp1a.get_traj(i))
    onset = exp1a.info['goal_onset'][i]
    i_subj = exp1a.info['subj_id'][i] 
    plt.plot(vals)
    if i_subj == 0:
        plt.plot(traj_speed(np.array(exp1a.get_traj(i))[:,0:2], exp1a.Hz))
        

